# Chapter 13: Rendering Strategies

Next.js offers multiple rendering strategies that allow you to optimize your application for performance, SEO, and user experience. Understanding when and how to use each strategy is crucial for building production-ready applications that balance server load, build times, and interactivity.

By the end of this chapter, you'll understand the differences between Static Site Generation, Server-Side Rendering, Incremental Static Regeneration, and Client-Side Rendering. You'll learn how to implement hybrid approaches and choose the optimal strategy for each page in your application.

## 13.1 Static Site Generation (SSG)

Static Site Generation renders pages at build time, creating highly performant HTML files that can be cached and served from a CDN.

### How SSG Works in Next.js

In the App Router, pages are statically generated by default when they don't use dynamic data fetching methods or dynamic route parameters without `generateStaticParams`.

```typescript
// app/about/page.tsx
// This page is statically generated at build time
export default function AboutPage() {
  return (
    <div className="container mx-auto py-12">
      <h1 className="text-4xl font-bold mb-6">About Us</h1>
      <p className="text-lg text-gray-700">
        We are a company dedicated to building amazing web experiences.
      </p>
    </div>
  );
}
```

### Static Generation with Data

Fetch data at build time using standard `fetch` with caching:

```typescript
// app/blog/posts/page.tsx
import { PostCard } from '@/components/post-card';

async function getPosts() {
  // This fetch is cached by default and runs at build time
  const res = await fetch('https://api.example.com/posts', {
    cache: 'force-cache', // Explicitly cache (default in SSG)
  });
  
  if (!res.ok) {
    throw new Error('Failed to fetch posts');
  }
  
  return res.json();
}

export default async function BlogPostsPage() {
  const posts = await getPosts();
  
  return (
    <div className="container mx-auto py-8">
      <h1 className="text-3xl font-bold mb-8">Blog Posts</h1>
      <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-6">
        {posts.map((post) => (
          <PostCard key={post.id} post={post} />
        ))}
      </div>
    </div>
  );
}
```

### generateStaticParams for Dynamic Routes

For dynamic routes, use `generateStaticParams` to specify which pages to generate at build time:

```typescript
// app/blog/[slug]/page.tsx
import { notFound } from 'next/navigation';

interface PageProps {
  params: { slug: string };
}

// Generate static paths at build time
export async function generateStaticParams() {
  const posts = await fetch('https://api.example.com/posts').then((res) => 
    res.json()
  );
  
  return posts.map((post: { slug: string }) => ({
    slug: post.slug,
  }));
}

async function getPost(slug: string) {
  const res = await fetch(`https://api.example.com/posts/${slug}`, {
    cache: 'force-cache',
  });
  
  if (!res.ok) {
    return null;
  }
  
  return res.json();
}

export default async function BlogPostPage({ params }: PageProps) {
  const post = await getPost(params.slug);
  
  if (!post) {
    notFound();
  }
  
  return (
    <article className="container mx-auto py-8 max-w-3xl">
      <h1 className="text-4xl font-bold mb-4">{post.title}</h1>
      <div className="prose lg:prose-xl">
        {post.content}
      </div>
    </article>
  );
}
```

## 13.2 Server-Side Rendering (SSR)

Server-Side Rendering generates HTML on each request, ensuring fresh data and personalized content.

### Implementing SSR

In the App Router, SSR is triggered when you opt out of caching or use dynamic data fetching:

```typescript
// app/dashboard/page.tsx
import { cookies } from 'next/headers';

// Force dynamic rendering (SSR)
export const dynamic = 'force-dynamic';

async function getUserData(userId: string) {
  // This runs on the server for each request
  const res = await fetch(`https://api.example.com/users/${userId}`, {
    cache: 'no-store', // Don't cache, fetch fresh data
  });
  
  return res.json();
}

export default async function DashboardPage() {
  const cookieStore = cookies();
  const userId = cookieStore.get('user-id')?.value;
  
  if (!userId) {
    return <div>Please log in</div>;
  }
  
  const userData = await getUserData(userId);
  
  return (
    <div className="container mx-auto py-8">
      <h1 className="text-3xl font-bold mb-6">Dashboard</h1>
      <div className="bg-white shadow rounded-lg p-6">
        <h2 className="text-xl font-semibold mb-2">
          Welcome, {userData.name}
        </h2>
        <p className="text-gray-600">
          Last login: {new Date(userData.lastLogin).toLocaleString()}
        </p>
      </div>
    </div>
  );
}
```

### Dynamic Data Fetching Patterns

Control caching behavior to determine when SSR occurs:

```typescript
// lib/data.ts
export async function getRealtimeData() {
  // Always fetch fresh data (SSR)
  const res = await fetch('https://api.example.com/realtime', {
    cache: 'no-store',
  });
  return res.json();
}

export async function getUserSpecificData(userId: string) {
  // Revalidate every 0 seconds (SSR with revalidation hint)
  const res = await fetch(`https://api.example.com/user/${userId}/data`, {
    next: { revalidate: 0 },
  });
  return res.json();
}

export async function getSearchResults(query: string) {
  // Dynamic rendering based on search params
  const res = await fetch(
    `https://api.example.com/search?q=${encodeURIComponent(query)}`,
    { cache: 'no-store' }
  );
  return res.json();
}
```

## 13.3 Incremental Static Regeneration (ISR)

ISR allows you to update static content after build time without rebuilding the entire site.

### Basic ISR Implementation

Use the `revalidate` option to specify how often pages should be regenerated:

```typescript
// app/products/page.tsx
async function getProducts() {
  const res = await fetch('https://api.example.com/products', {
    next: { revalidate: 60 }, // Regenerate every 60 seconds
  });
  return res.json();
}

export default async function ProductsPage() {
  const products = await getProducts();
  
  return (
    <div className="container mx-auto py-8">
      <h1 className="text-3xl font-bold mb-8">Products</h1>
      <div className="grid grid-cols-1 md:grid-cols-3 gap-6">
        {products.map((product) => (
          <ProductCard key={product.id} product={product} />
        ))}
      </div>
      <p className="text-sm text-gray-500 mt-8">
        Last updated: {new Date().toLocaleTimeString()}
      </p>
    </div>
  );
}
```

### ISR with Dynamic Routes

Combine `generateStaticParams` with revalidation:

```typescript
// app/products/[id]/page.tsx
interface PageProps {
  params: { id: string };
}

export const revalidate = 3600; // Regenerate every hour

export async function generateStaticParams() {
  // Pre-generate top 100 products at build time
  const products = await fetch(
    'https://api.example.com/products?limit=100'
  ).then((res) => res.json());
  
  return products.map((product: { id: string }) => ({
    id: product.id,
  }));
}

async function getProduct(id: string) {
  const res = await fetch(`https://api.example.com/products/${id}`, {
    next: { revalidate: 3600 },
  });
  
  if (!res.ok) return null;
  return res.json();
}

export default async function ProductPage({ params }: PageProps) {
  const product = await getProduct(params.id);
  
  if (!product) {
    notFound();
  }
  
  return (
    <div className="container mx-auto py-8">
      <h1 className="text-3xl font-bold mb-4">{product.name}</h1>
      <p className="text-2xl text-green-600 mb-6">${product.price}</p>
      <p className="text-gray-700">{product.description}</p>
    </div>
  );
}
```

### On-Demand Revalidation

Trigger revalidation programmatically via API routes:

```typescript
// app/api/revalidate/route.ts
import { revalidatePath, revalidateTag } from 'next/cache';
import { NextRequest, NextResponse } from 'next/server';

export async function POST(request: NextRequest) {
  const secret = request.headers.get('x-revalidate-secret');
  
  if (secret !== process.env.REVALIDATE_SECRET) {
    return NextResponse.json(
      { error: 'Invalid secret' },
      { status: 401 }
    );
  }
  
  try {
    const { path, tag } = await request.json();
    
    if (path) {
      revalidatePath(path);
      return NextResponse.json({
        revalidated: true,
        path,
        now: Date.now(),
      });
    }
    
    if (tag) {
      revalidateTag(tag);
      return NextResponse.json({
        revalidated: true,
        tag,
        now: Date.now(),
      });
    }
    
    return NextResponse.json(
      { error: 'Path or tag required' },
      { status: 400 }
    );
  } catch (error) {
    return NextResponse.json(
      { error: 'Revalidation failed' },
      { status: 500 }
    );
  }
}
```

## 13.4 Client-Side Rendering (CSR)

Client-Side Rendering fetches data and renders content in the browser, ideal for highly interactive pages.

### When to Use CSR

Use CSR for user-specific dashboards, real-time data, or highly interactive features:

```typescript
// app/dashboard/analytics/page.tsx
'use client';

import { useEffect, useState } from 'react';
import { LineChart, Line, XAxis, YAxis, Tooltip } from 'recharts';

interface AnalyticsData {
  date: string;
  views: number;
  clicks: number;
}

export default function AnalyticsDashboard() {
  const [data, setData] = useState<AnalyticsData[]>([]);
  const [loading, setLoading] = useState(true);
  const [timeRange, setTimeRange] = useState('7d');

  useEffect(() => {
    async function fetchData() {
      try {
        const res = await fetch(`/api/analytics?range=${timeRange}`);
        const json = await res.json();
        setData(json.data);
      } catch (error) {
        console.error('Failed to fetch analytics:', error);
      } finally {
        setLoading(false);
      }
    }
    
    fetchData();
  }, [timeRange]);

  if (loading) {
    return <div className="p-8">Loading analytics...</div>;
  }

  return (
    <div className="container mx-auto py-8">
      <div className="flex justify-between items-center mb-8">
        <h1 className="text-3xl font-bold">Analytics Dashboard</h1>
        <select
          value={timeRange}
          onChange={(e) => setTimeRange(e.target.value)}
          className="border rounded p-2"
        >
          <option value="24h">Last 24 Hours</option>
          <option value="7d">Last 7 Days</option>
          <option value="30d">Last 30 Days</option>
        </select>
      </div>
      
      <div className="bg-white p-6 rounded-lg shadow">
        <LineChart width={800} height={400} data={data}>
          <XAxis dataKey="date" />
          <YAxis />
          <Tooltip />
          <Line type="monotone" dataKey="views" stroke="#8884d8" />
          <Line type="monotone" dataKey="clicks" stroke="#82ca9d" />
        </LineChart>
      </div>
    </div>
  );
}
```

### Hybrid Approach: Static Shell with CSR

Combine static rendering with client-side data fetching:

```typescript
// app/profile/page.tsx
import { Suspense } from 'react';
import { ProfileSkeleton } from '@/components/skeletons';
import { ProfileHeader } from '@/components/profile-header';

// Static header generated at build time
export default function ProfilePage() {
  return (
    <div className="container mx-auto py-8">
      <h1 className="text-3xl font-bold mb-8">User Profile</h1>
      
      {/* Static content */}
      <div className="bg-gray-50 p-6 rounded-lg mb-6">
        <h2 className="text-lg font-semibold mb-2">About This Page</h2>
        <p className="text-gray-600">
          Your profile information is loaded dynamically for the most 
          up-to-date data.
        </p>
      </div>
      
      {/* Client-side fetched content */}
      <Suspense fallback={<ProfileSkeleton />}>
        <ProfileHeader />
      </Suspense>
      
      {/* More client components */}
      <Suspense fallback={<div>Loading activity...</div>}>
        <UserActivity />
      </Suspense>
    </div>
  );
}

// components/profile-header.tsx
'use client';

import { useEffect, useState } from 'react';

export function ProfileHeader() {
  const [user, setUser] = useState(null);
  
  useEffect(() => {
    fetch('/api/user/profile')
      .then(res => res.json())
      .then(data => setUser(data));
  }, []);
  
  if (!user) return <ProfileSkeleton />;
  
  return (
    <div className="bg-white shadow rounded-lg p-6">
      <h2 className="text-2xl font-bold">{user.name}</h2>
      <p className="text-gray-600">{user.email}</p>
    </div>
  );
}
```

## 13.5 Hybrid Rendering Approaches

Modern Next.js applications combine multiple rendering strategies for optimal performance.

### Route Segment Configurations

Control rendering at the route level:

```typescript
// app/layout.tsx or page.tsx
// Force dynamic rendering for the entire route
export const dynamic = 'force-dynamic';

// Force static generation
export const dynamic = 'force-static';

// Auto - let Next.js decide (default)
export const dynamic = 'auto';
```

### Granular Caching Strategies

Apply different strategies to different data sources:

```typescript
// app/complex-page/page.tsx
import { Suspense } from 'react';

// Component 1: Static data (SSG)
async function StaticContent() {
  const data = await fetch('https://api.example.com/static-content', {
    cache: 'force-cache', // Build-time fetch
  });
  const content = await data.json();
  
  return <div>{content.text}</div>;
}

// Component 2: Cached but revalidatable (ISR)
async function SemiStaticContent() {
  const data = await fetch('https://api.example.com/news', {
    next: { revalidate: 3600 }, // Hourly updates
  });
  const news = await data.json();
  
  return <NewsList items={news} />;
}

// Component 3: Dynamic data (SSR)
async function DynamicContent() {
  const data = await fetch('https://api.example.com/weather', {
    cache: 'no-store', // Fresh data every request
  });
  const weather = await data.json();
  
  return <WeatherWidget data={weather} />;
}

// Main page combining all strategies
export default function ComplexPage() {
  return (
    <div className="container mx-auto py-8">
      <h1 className="text-3xl font-bold mb-8">Hybrid Example</h1>
      
      <section className="mb-8">
        <h2 className="text-xl font-semibold mb-4">Static Content</h2>
        <StaticContent />
      </section>
      
      <section className="mb-8">
        <h2 className="text-xl font-semibold mb-4">Hourly Updates</h2>
        <Suspense fallback={<div>Loading news...</div>}>
          <SemiStaticContent />
        </Suspense>
      </section>
      
      <section>
        <h2 className="text-xl font-semibold mb-4">Live Weather</h2>
        <Suspense fallback={<div>Loading weather...</div>}>
          <DynamicContent />
        </Suspense>
      </section>
    </div>
  );
}
```

## 13.6 Choosing the Right Strategy

Selecting the appropriate rendering strategy depends on your specific use case.

### Decision Matrix

| Strategy | Use Case | SEO | Performance | Data Freshness |
|----------|----------|-----|-------------|----------------|
| **SSG** | Marketing pages, blogs, documentation | Excellent | Excellent | Build-time only |
| **ISR** | E-commerce products, news sites | Excellent | Excellent | Configurable |
| **SSR** | User dashboards, personalized content | Good | Good | Real-time |
| **CSR** | Admin panels, real-time apps | Poor (use SSG shell) | Good after initial load | Real-time |

### Implementation Guidelines

```typescript
// Examples by use case

// 1. Marketing Landing Page → SSG
// app/page.tsx (no special config needed)
export default function LandingPage() {
  return <div>Static content</div>;
}

// 2. E-commerce Product → ISR
// app/shop/[id]/page.tsx
export const revalidate = 60;

export async function generateStaticParams() {
  // Generate popular products at build
  return topProducts.map(p => ({ id: p.id }));
}

// 3. User Dashboard → SSR
// app/dashboard/page.tsx
export const dynamic = 'force-dynamic';

export default async function Dashboard() {
  const data = await getUserData(); // Fresh data per request
  return <DashboardView data={data} />;
}

// 4. Real-time Chat → CSR with SSG Shell
// app/chat/page.tsx
export default function ChatPage() {
  return (
    <>
      <StaticHeader /> {/* SSG */}
      <ChatClient />   {/* CSR */}
    </>
  );
}
```

## 13.7 Performance Implications

Understanding the performance characteristics of each strategy helps optimize your application.

### Build Time vs Request Time

```typescript
// Build-time considerations
// app/generate-heavy/page.tsx

// This will slow down builds if there are thousands of items
export async function generateStaticParams() {
  const items = await fetchAllItems(); // 10,000 items?
  
  // Consider limiting build-time generation
  return items.slice(0, 100).map(item => ({
    id: item.id,
  }));
  
  // Remaining items use ISR on first visit
}

export const revalidate = 86400; // 24 hours
```

### Streaming and Partial Prerendering

Use streaming to improve perceived performance:

```typescript
// app/streaming/page.tsx
import { Suspense } from 'react';
import { unstable_noStore as noStore } from 'next/cache';

async function SlowComponent() {
  noStore(); // Opt out of caching
  
  await new Promise(resolve => setTimeout(resolve, 2000));
  
  return <div>Slow data loaded</div>;
}

async function FastComponent() {
  return <div>Fast data loaded immediately</div>;
}

export default function StreamingPage() {
  return (
    <div className="container mx-auto py-8">
      <h1 className="text-3xl font-bold mb-8">Streaming Example</h1>
      
      {/* This renders immediately */}
      <FastComponent />
      
      {/* This streams in when ready */}
      <Suspense fallback={<div className="animate-pulse">Loading...</div>}>
        <SlowComponent />
      </Suspense>
    </div>
  );
}
```

### Performance Monitoring

Track your rendering performance:

```typescript
// lib/performance.ts
export function logRenderTime(
  componentName: string, 
  startTime: number
) {
  const duration = performance.now() - startTime;
  
  if (duration > 100) {
    console.warn(
      `Slow render detected: ${componentName} took ${duration.toFixed(2)}ms`
    );
  }
}

// Usage in components
export default async function DataPage() {
  const startTime = performance.now();
  
  const data = await fetchData();
  
  logRenderTime('DataPage', startTime);
  
  return <div>{data.content}</div>;
}
```

## Key Takeaways from Chapter 13

1. **Static Site Generation (SSG)**: Pages are rendered at build time, providing excellent performance and SEO. Use for content that doesn't change often like marketing pages, blogs, and documentation. Use `generateStaticParams` for dynamic routes.

2. **Server-Side Rendering (SSR)**: Pages are rendered on the server for each request, ensuring fresh data and personalization. Use `export const dynamic = 'force-dynamic'` or `cache: 'no-store'` to enable SSR for user-specific content.

3. **Incremental Static Regeneration (ISR)**: Combines the benefits of SSG with the ability to update content after build. Use the `revalidate` option to specify update intervals, and implement on-demand revalidation for immediate updates when content changes.

4. **Client-Side Rendering (CSR)**: Data fetching occurs in the browser using React hooks like `useEffect`. Ideal for highly interactive features, real-time data, and user-specific dashboards that don't need SEO optimization.

5. **Hybrid Approaches**: Modern Next.js applications mix strategies—using SSG for the layout shell, ISR for semi-static content, and CSR for interactive elements. Use `Suspense` boundaries to manage loading states across different rendering modes.

6. **Choosing the Right Strategy**: Consider SEO requirements, data freshness needs, build time constraints, and user experience. Marketing content benefits from SSG/ISR, while personalized dashboards need SSR/CSR.

7. **Performance Implications**: SSG provides the fastest page loads but stale data. SSR provides fresh data but higher server load. ISR balances both but adds complexity. Monitor Core Web Vitals and use streaming to optimize perceived performance.

## Coming Up Next

**Chapter 14: Middleware & Proxy**

Now that you understand rendering strategies, it's time to learn how to intercept and modify requests before they reach your pages. In Chapter 14, we'll explore Middleware—a powerful feature for authentication, redirects, A/B testing, and the new Proxy configuration in Next.js 16. You'll learn how to run code at the edge and optimize request handling across your entire application.